In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
path='s:\\research\\ORSP\\INFO_CTR\\0_Reporting\\Predictive_Analytics_Project\\Core_Facilities_Data\\'
file='NUFS-PI-Crosswalk.xlsx'
df=pd.read_excel(path+file)

In [3]:
df.rename(columns={'Sponsored Award Number': 'SponsoredAwardNumber','Originating Spon Code Level 2': 'OriginatingSponCodeLevel2'}, inplace=True)

In [4]:
def fix_nih(df):
   df['InterimNIHStep']=(((df['SponsoredAwardNumber'].str.split('//').str[-1]).str.split('-').str[0]).str.replace(' ','')).str.lstrip('123456789')
   df['CleanSponsorNumber']=df['InterimNIHStep'].str.split('/').str[0]
   return(df)

In [8]:
def fix_nsf(df):
    df['InterimNSFStep']=((df['SponsoredAwardNumber'].str.split('/').str[0]).str.split('\(')).str[0]
    df['NSFDirectorate']=((df['InterimNSFStep'].str.split('-')).str[0])
    df['InterimNSFStep']=((df['InterimNSFStep'].str.split('-')).str[-1])
    df['CleanSponsorNumber']=df['InterimNSFStep'].str.strip()
    return(df)

In [9]:
def fix_dod(df):
    df['InterimDODStep']=(df['SponsoredAwardNumber'].str.split('//').str[-1])
    df['CleanSponsorNumber']=(df['InterimDODStep'].str.split('/').str[0]).str.split('\(').str[0]
    return(df)

In [10]:
def use_fain(df):
    df['CleanSponsorNumber']=(df['Fain'])
    return(df)

In [11]:
def use_existing(df):
    df['CleanSponsorNumber']=(df['SponsoredAwardNumber'])
    return(df)

In [12]:
has_FAIN=df.Fain.notnull()
no_FAIN=df.Fain.isnull()
is_NIH=df.OriginatingSponCodeLevel2 == 'NIH'
is_DOD=df.OriginatingSponCodeLevel2 == 'DOD'
is_NSF=df.OriginatingSponCodeLevel2 == 'NSF'



In [13]:
df_fain=df[has_FAIN]
df_noFain=df[no_FAIN]
df_nih=df_noFain[is_NIH]
df_dod=df_noFain[is_DOD]
df_nsf=df_noFain[is_NSF]
df_other=df_noFain[~df.OriginatingSponCodeLevel2.isin(['NIH','DOD','NSF'])]
   

C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [14]:
assert(len(df_noFain)==len(df_nih)+len(df_dod)+len(df_nsf)+len(df_other))
assert(len(df_noFain)+len(df_fain)==len(df))



In [15]:
df_fain=use_fain(df_fain)
df_nih=fix_nih(df_nih)
df_dod=fix_dod(df_dod)
df_nsf=fix_nsf(df_nsf)
df_other=use_existing(df_other)


C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [16]:
df_clean=pd.concat([df_nih,df_dod,df_nsf,df_other,df_fain])
df_clean.head()


C:\Users\ksg181\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,CleanSponsorNumber,Employee ID,Fain,Full Name,Institution Number,InterimDODStep,InterimNIHStep,InterimNSFStep,NSFDirectorate,Net ID,OriginatingSponCodeLevel2,PeopleSoft Project ID,SponsoredAwardNumber
9817,costshare,1049927.0,NaN,Vadim Backman,PROJ0001621,NaN,costshare,NaN,NaN,vba990,NIH,60012293,cost share
9818,costshare,1023181.0,NaN,Annelise E Barron,PROJ0001622,NaN,costshare,NaN,NaN,aba461,NIH,60012294,cost share
9819,costshare,1052365.0,NaN,Teri W Odom,PROJ0001623,NaN,costshare,NaN,NaN,two626,NIH,60012295,cost share
9820,costshare,1052365.0,NaN,Teri W Odom,PROJ0001624,NaN,costshare,NaN,NaN,two626,NIH,60012296,cost share
9821,costshare,1002860.0,NaN,Chad A Mirkin,PROJ0001625,NaN,costshare,NaN,NaN,cam493,NIH,60012297,cost share


In [24]:
df_clean.drop('InterimStep',axis=1)

,CleanSponsorNumber,Employee ID,Fain,Full Name,Institution Number,Net ID,OriginatingSponCodeLevel2,PeopleSoft Project ID,SponsoredAwardNumber
9817,costshare,1049927.0,NaN,Vadim Backman,PROJ0001621,vba990,NIH,60012293,cost share
9818,costshare,1023181.0,NaN,Annelise E Barron,PROJ0001622,aba461,NIH,60012294,cost share
9819,costshare,1052365.0,NaN,Teri W Odom,PROJ0001623,two626,NIH,60012295,cost share
9820,costshare,1052365.0,NaN,Teri W Odom,PROJ0001624,two626,NIH,60012296,cost share
9821,costshare,1002860.0,NaN,Chad A Mirkin,PROJ0001625,cam493,NIH,60012297,cost share
9822,costshare,1028426.0,NaN,Sui Huang,PROJ0001626,shu683,NIH,60012298,cost share
9823,R01GM52329,1018680.0,NaN,Robert C MacDonald,CNV0026691 - S1,macd,NIH,60012300,3 R01 GM52329-07S1
9824,R01GM064715,1034628.0,NaN,Ishwar Radhakrishnan,CNV0029311 - S1,ira124,NIH,60012301,3 R01 GM064715-02S1
9825,R01EY006669,1008747.0,NaN,John B Troy,CNV0018636 - S1,jbtroy,NIH,60012302,3 R01 EY006669-12S1
9826,R01HD039406,1031152.0,NaN,David A Brown,SP0003465,dbr524,NIH,60012303,5 R01 HD039406-04 (Rev.9/3/03)


In [19]:
writer = pd.ExcelWriter(path + 'ParsedOutput.xlsx', engine='xlsxwriter')

In [20]:
df_clean.to_excel(writer)